### Importing libraries that will be useful :-

In [1]:
import pandas as pd 
import numpy as np
from pprint import pprint
import warnings 
warnings.filterwarnings('ignore')
import datetime
from dateutil import relativedelta
import re

# Question 4 solution : 

Reading data from districts.csv file i.e covid data :-

In [2]:
covid_data = pd.read_csv('districts.csv',usecols=['Date','State','District','Confirmed','Recovered','Deceased'])
# covid_data

In [3]:
covid_data['Active'] = covid_data['Confirmed'] - covid_data['Recovered'] - covid_data['Deceased']
covid_data= covid_data[0:307246][:] # Taking data only upto 14th August.

Since some district names are same across states, so first lets create district ID for every district and then drop state and district coloumns as they will no longer be required.

In [4]:
covid_data["District_ID"] = covid_data["State"] + '_' + covid_data["District"]
# covid_data = covid_data.drop(columns=["State","District"])
covid_data1 = covid_data[['District_ID','Date','Active']].copy()
# covid_data1

# Finding peak for every district :-

## Finding weeks that had peek i.e highest no of cases 

In [5]:
covid_data1['District_ID'] = covid_data1['District_ID'] + '_' + covid_data1['Date']
covid_data1 = covid_data1.drop(columns=["Date"])
covid_data1 = covid_data1.set_index(['District_ID'])
# covid_data1

In [6]:
district_ids = covid_data[covid_data['Date']=='2021-08-14']['District_ID'].copy()
district_ids = district_ids.reset_index()
district_ids = district_ids[['District_ID']]
# district_ids

In [7]:
def case_this_day(district,date):
    Id = district + '_' + str(date)
    try:
        cases = covid_data1.at[Id,'Active']
    except:
        cases = 0
    if (cases>=0):
        return cases
    else:
        return 0

In [8]:
def return_date(date_case):
    date_case = str(date_case)
    match_str = re.search(r'\d{4}-\d{2}-\d{2}', date_case)
    res = datetime.datetime.strptime(match_str.group(), '%Y-%m-%d').date()
    return str(res)

In [9]:
weekly_data = pd.DataFrame()
list = ['districtid','timeid','cases']
weekly_data1 = pd.DataFrame(columns=list)
start_date1 = datetime.datetime.strptime('2020/04/26','%Y/%m/%d')
start_date2 = datetime.datetime.strptime('2020/04/30','%Y/%m/%d')
end_date = datetime.datetime.strptime('2021/08/14','%Y/%m/%d')
for i in district_ids.index:
    w_no = 1
    flag = 0
    district = district_ids.iloc[i]['District_ID']
    while start_date1 < end_date and start_date2 < end_date:
        if flag == 0:
            next_date1 = start_date1 + relativedelta.relativedelta(weeks=1) - datetime.timedelta(days=1)
            cases = case_this_day(district,return_date(next_date1))
            weekly_data1['districtid'] = [district]
            weekly_data1['timeid'] = w_no
            weekly_data1['cases'] = cases
            weekly_data = weekly_data.append(weekly_data1, ignore_index=True)
            w_no += 1
            start_date1 += relativedelta.relativedelta(weeks=1)
            flag = 1
        else :
            next_date2 = start_date2 + relativedelta.relativedelta(weeks=1) - datetime.timedelta(days=1)
            cases = case_this_day(district,return_date(next_date2))
            weekly_data1['districtid'] = [district]
            weekly_data1['timeid'] = w_no
            weekly_data1['cases'] = cases
            weekly_data = weekly_data.append(weekly_data1, ignore_index=True)
            w_no += 1
            start_date2 += relativedelta.relativedelta(weeks=1)
            flag = 0
    start_date1 = datetime.datetime.strptime('2020/04/26','%Y/%m/%d')
    start_date2 = datetime.datetime.strptime('2020/04/30','%Y/%m/%d')
# weekly_data

In [21]:
weekly_data

,districtid,timeid,cases
0,Andaman and Nicobar Islands_Unknown,1,7
1,Andaman and Nicobar Islands_Unknown,2,1
2,Andaman and Nicobar Islands_Unknown,3,0
3,Andaman and Nicobar Islands_Unknown,4,0
4,Andaman and Nicobar Islands_Unknown,5,0
...,...,...,...
88825,West Bengal_Uttar Dinajpur,131,127
88826,West Bengal_Uttar Dinajpur,132,132
88827,West Bengal_Uttar Dinajpur,133,129
88828,West Bengal_Uttar Dinajpur,134,126


In [10]:
list = ['districtid', 'wave1_weekid', 'wave2_weekid']
temp_d2 = pd.DataFrame(columns=list)

In [22]:
weeklydata = weekly_data.copy()
temp_d1 = pd.DataFrame()
weekly_output = pd.DataFrame()
gb = weeklydata.groupby(['districtid'])
for x in gb.groups:
    temp_d1 = gb.get_group(x)
    temp_d1 = temp_d1.reset_index()
    cases_array = temp_d1[["cases"]].to_numpy()
    wave = np.split(cases_array, [68])
    max1 = np.argmax(wave[0]) 
    max2 = 68 + np.argmax(wave[1])
#     print(temp_d1.at[max1,'timeid']," ", temp_d1.at[max2,'timeid'])
    temp_d2['districtid'] = [temp_d1.at[max1,'districtid']]
    temp_d2['wave1_weekid'] = temp_d1.at[max1,'timeid']
    temp_d2['wave2_weekid'] = temp_d1.at[max2,'timeid']
    weekly_output = weekly_output.append(temp_d2, ignore_index=True)
#     break
    
# temp_d1["timeid"] = temp_d1["timeid"].astype(str)
# temp_d1['timeid'] = 'Week_' + temp_d1['timeid']
# temp_d1
weekly_output

,districtid,wave1_weekid,wave2_weekid
0,Andaman and Nicobar Islands_Unknown,31,111
1,Andhra Pradesh_Anantapur,27,109
2,Andhra Pradesh_Chittoor,33,111
3,Andhra Pradesh_East Godavari,36,112
4,Andhra Pradesh_Foreign Evacuees,18,69
...,...,...,...
653,West Bengal_Purba Bardhaman,52,107
654,West Bengal_Purba Medinipur,55,111
655,West Bengal_Purulia,41,104
656,West Bengal_South 24 Parganas,53,111


## Finding months that had peek i.e highest no of cases 

In [12]:
monthly_data = pd.DataFrame()
list = ['districtid','timeid','cases']
monthly_data1 = pd.DataFrame(columns=list)
start_date = datetime.datetime.strptime('2020/04/26','%Y/%m/%d')
end_date = datetime.datetime.strptime('2021/08/14','%Y/%m/%d')
for i in district_ids.index:
    m_no = 1
    district = district_ids.iloc[i]['District_ID']
    while start_date < end_date:
        next_date = start_date + relativedelta.relativedelta(months=1) - datetime.timedelta(days=1)
        cases = case_this_day(district,return_date(next_date)) #- case_this_day(district,return_date(start_date))
        monthly_data1['districtid'] = [district]
        monthly_data1['timeid'] = m_no
        monthly_data1['cases'] = cases
        monthly_data = monthly_data.append(monthly_data1, ignore_index=True)
        m_no += 1
        start_date += relativedelta.relativedelta(months=1)
    start_date = datetime.datetime.strptime('2020/04/26','%Y/%m/%d')
#     break
# monthly_data

In [13]:
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     display(monthly_data)

In [14]:
list1 = ['districtid', 'wave1_monthid', 'wave2_monthid']
temp_d3 = pd.DataFrame(columns=list1)
# temp_d3

In [15]:
monthlydata = monthly_data.copy()
temp_d12 = pd.DataFrame()
monthly_output = pd.DataFrame()
gb = monthlydata.groupby(['districtid'])
for x in gb.groups:
    temp_d12 = gb.get_group(x)
    temp_d12 = temp_d12.reset_index()
    cases_array1 = temp_d12[["cases"]].to_numpy()
    wave = np.split(cases_array1, [8])
    max1 = np.argmax(wave[0]) 
    max2 = 8 + np.argmax(wave[1])
#     print(temp_d1.at[max1,'timeid']," ", temp_d1.at[max2,'timeid'])
    temp_d3['districtid'] = [temp_d12.at[max1,'districtid']]
    temp_d3['wave1_monthid'] = temp_d12.at[max1,'timeid']
    temp_d3['wave2_monthid'] = temp_d12.at[max2,'timeid']
    monthly_output = monthly_output.append(temp_d3, ignore_index=True)
#     break
    
# temp_d1["timeid"] = temp_d1["timeid"].astype(str)
# temp_d1['timeid'] = 'Week_' + temp_d1['timeid']
# temp_d1
# monthly_output

In [16]:
data_sample = pd.merge(weekly_output,monthly_output,left_on='districtid', right_on='districtid')
# data_sample

In [17]:
data_sample.to_csv('district-peaks.csv', index=False)

In [18]:
# district-peaks.csv
data_sample

,districtid,wave1_weekid,wave2_weekid,wave1_monthid,wave2_monthid
0,Andaman and Nicobar Islands_Unknown,31,111,4,13
1,Andhra Pradesh_Anantapur,27,109,4,13
2,Andhra Pradesh_Chittoor,33,111,4,13
3,Andhra Pradesh_East Godavari,36,112,4,13
4,Andhra Pradesh_Foreign Evacuees,18,69,2,9
...,...,...,...,...,...
653,West Bengal_Purba Bardhaman,52,107,6,13
654,West Bengal_Purba Medinipur,55,111,4,13
655,West Bengal_Purulia,41,104,6,12
656,West Bengal_South 24 Parganas,53,111,6,13


## Finding peak for every state :-

In [19]:
covid_data_copy = covid_data.copy()

In [20]:
covid_data_copy['Active'] = covid_data_copy['Confirmed'] - covid_data_copy['Recovered'] - covid_data_copy['Deceased']
covid_data_copy= covid_data_copy[0:307246][:] # Taking data only upto 14th August.
covid_data_copy = covid_data_copy.drop(columns=["District","Confirmed","Recovered","Deceased"])
covid_data_copy = covid_data_copy.groupby(['Date','State']).agg({'Active':'sum'}).reset_index()  
# covid_data_copy = covid_data_copy.sort_values('District',ignore_index=True)
# covid_data_copy

# Q4 Completed..:)